In [2]:
# SchizApathy Mini-Project: Functional Connectivity in Apathy-Linked Brain Circuits

# --- SETUP ---
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import gzip
from nilearn import datasets, image, plotting, input_data
import nibabel as nib

# --- LOAD SUBJECT IDS ---
# Read list of subject IDs from a text file
with open("/home/cgntlab4/Downloads/4197885/downloaded_subjects.txt") as f:
    subject_ids = [line.strip() for line in f if line.strip()]

# --- SET BASE DIRECTORY ---
base_dir = '/home/cgntlab4/Downloads/4197885'

# --- LOAD PHENOTYPIC DATA ---
# Load subject labels (SZ vs HC)
pheno_df = pd.read_csv(os.path.join(base_dir, 'phenotypic_data.tsv'), sep='\t')
pheno_df['StrID'] = pheno_df['ID'].astype(str).str.zfill(7)
pheno_df = pheno_df[pheno_df['StrID'].isin(subject_ids)]
sz_subjects = set(pheno_df[pheno_df['Subject Type'] == 'Patient']['StrID'])
hc_subjects = set(pheno_df[pheno_df['Subject Type'] == 'Control']['StrID'])

# --- LOAD ATLAS ---
# Load Schaefer 100 ROI atlas and decode labels
atlas = datasets.fetch_atlas_schaefer_2018(n_rois=100, yeo_networks=7, resolution_mm=2)
atlas_img = atlas['maps']
atlas_labels = [label.decode('utf-8') if isinstance(label, bytes) else label for label in atlas['labels']]

# --- FUNCTION TO LOAD CONFOUNDS ---
def load_confounds(confound_path):
    try:
        return pd.read_csv(confound_path, sep='\t')
    except Exception as e:
        print(f"Error reading confounds for {confound_path}: {e}")
        return None

# --- FUNCTION TO PROCESS INDIVIDUAL SUBJECTS ---
def process_subject(sub_id):
    bold_path = os.path.join(base_dir, f'fmri_{sub_id}.nii.gz')
    confound_path = os.path.join(base_dir, f'fmri_{sub_id}.tsv.gz')

    if not os.path.exists(bold_path) or not os.path.exists(confound_path):
        print(f"Missing files for subject {sub_id}")
        return None

    confounds_df = load_confounds(confound_path)
    if confounds_df is None:
        return None

    confounds_numeric = confounds_df.select_dtypes(include=[np.number]).fillna(0)
    subject_img = nib.load(bold_path)
    resampled_atlas = image.resample_to_img(atlas_img, subject_img, interpolation='nearest')

    masker = input_data.NiftiLabelsMasker(
        labels_img=resampled_atlas,
        standardize=True,
        memory='nilearn_cache')

    return masker.fit_transform(subject_img, confounds=confounds_numeric)

# --- EXTRACT TIME SERIES FOR ALL SUBJECTS ---
subject_data = {}
for sub in subject_ids:
    data = process_subject(sub)
    if data is not None:
        print(f"✔ Processed subject {sub}: shape = {data.shape}")
        subject_data[sub] = data

# --- FUNCTIONAL CONNECTIVITY MATRICES ---
def compute_fc_matrix(timeseries):
    return np.corrcoef(timeseries.T) if timeseries.size > 0 else None

connectivity_matrices = {
    sub: compute_fc_matrix(ts) for sub, ts in subject_data.items() if compute_fc_matrix(ts) is not None
}

# --- GROUP AVERAGES ---
sz_matrices = [mat for sid, mat in connectivity_matrices.items() if sid in sz_subjects]
hc_matrices = [mat for sid, mat in connectivity_matrices.items() if sid in hc_subjects]

sz_avg = np.mean(sz_matrices, axis=0) if sz_matrices else np.array([])
hc_avg = np.mean(hc_matrices, axis=0) if hc_matrices else np.array([])
diff_matrix = sz_avg - hc_avg if sz_avg.size and hc_avg.size else np.array([])

# --- STATISTICAL TESTING ---
from scipy.stats import ttest_ind
if sz_matrices and hc_matrices:
    sz_array = np.stack(sz_matrices)
    hc_array = np.stack(hc_matrices)
    t_stats, p_vals = ttest_ind(sz_array, hc_array, axis=0)
    np.fill_diagonal(t_stats, 0)
    np.fill_diagonal(p_vals, 1)

    np.savetxt("t_stat_matrix.csv", t_stats, delimiter=",", header=",".join(atlas_labels), comments='')
    np.savetxt("p_val_matrix.csv", p_vals, delimiter=",", header=",".join(atlas_labels), comments='')

    sig_mask = p_vals < 0.05
    diff_sig = diff_matrix * sig_mask
    plt.figure(figsize=(10, 8))
    sns.heatmap(diff_sig, cmap='coolwarm', center=0)
    plt.title("Significant Differences (p < 0.05)")
    plt.tight_layout()
    plt.savefig("fc_significant_diff.png")
    plt.close()

# --- APATHY FOCUSED ANALYSIS ---
apathy_keywords = ['cingulate', 'frontal', 'orbitofrontal', 'insula', 'amygdala', 'OFC', 'PFC']
apathy_indices = [i for i, label in enumerate(atlas_labels) if any(k.lower() in label.lower() for k in apathy_keywords)]

if sz_matrices and hc_matrices and apathy_indices:
    apathy_pairs = [(i, j, t_stats[i, j], p_vals[i, j]) for i in apathy_indices for j in range(len(atlas_labels)) if i != j]
    top_apathy = sorted(apathy_pairs, key=lambda x: abs(x[2]), reverse=True)[:10]
    apathy_df = pd.DataFrame([{'ROI 1': atlas_labels[i], 'ROI 2': atlas_labels[j], 't-stat': t, 'p-val': p} for i, j, t, p in top_apathy])
    apathy_df.to_csv("apathy_top_differences.csv", index=False)

    for i, j, t, p in top_apathy:
        print(f"{atlas_labels[i]} ↔ {atlas_labels[j]} | t = {t:.3f}, p = {p:.4f}")

    from nilearn.plotting import plot_connectome
    example_img = nib.load(os.path.join(base_dir, f'fmri_{subject_ids[0]}.nii.gz'))
    resampled = image.resample_to_img(atlas_img, example_img, interpolation='nearest')
    coords = plotting.find_parcellation_cut_coords(labels_img=resampled)
    apathy_conn = np.zeros((len(atlas_labels), len(atlas_labels)))
    for i, j, _, _ in top_apathy:
        apathy_conn[i, j] = diff_matrix[i, j]
        apathy_conn[j, i] = diff_matrix[i, j]
    plot_connectome(apathy_conn, coords, output_file="apathy_connectome.png")

# --- PLOT GROUP AVERAGE MATRICES ---
def plot_and_save(matrix, title, filename):
    if matrix.size:
        plt.figure(figsize=(10, 8))
        sns.heatmap(matrix, cmap='coolwarm', center=0)
        plt.title(title)
        plt.tight_layout()
        plt.savefig(filename)
        plt.close()

plot_and_save(sz_avg, "Average FC - Schizophrenia", "fc_schizophrenia.png")
plot_and_save(hc_avg, "Average FC - Control", "fc_control.png")
plot_and_save(diff_matrix, "Difference FC (SZ - HC)", "fc_difference.png")


[get_dataset_dir] Dataset found in /home/cgntlab4/nilearn_data/schaefer_2018

/tmp/ipykernel_4090014/666536536.py:58: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  resampled_atlas = image.resample_to_img(atlas_img, subject_img, interpolation='nearest')
/home/cgntlab4/anaconda3/lib/python3.12/site-packages/nilearn/image/resampling.py:805: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  return resample_img(
/tmp/ipykernel_4090014/666536536.py:58: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  resampled_atlas = image.resample_to_img(atlas_img, subject_img, interpolation='nearest')
/home/cgntlab4/anaconda3/lib/python3.12/site-packages/nilearn/image/resampling.py:805: FutureWa

✔ Processed subject 0040000: shape = (150, 100)
✔ Processed subject 0040001: shape = (150, 100)
✔ Processed subject 0040002: shape = (150, 100)
✔ Processed subject 0040003: shape = (150, 100)
✔ Processed subject 0040004: shape = (150, 100)


/home/cgntlab4/anaconda3/lib/python3.12/site-packages/nilearn/image/resampling.py:805: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  return resample_img(
/tmp/ipykernel_4090014/666536536.py:58: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  resampled_atlas = image.resample_to_img(atlas_img, subject_img, interpolation='nearest')
/home/cgntlab4/anaconda3/lib/python3.12/site-packages/nilearn/image/resampling.py:805: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  return resam

✔ Processed subject 0040005: shape = (150, 100)
✔ Processed subject 0040006: shape = (150, 100)
✔ Processed subject 0040007: shape = (150, 100)
✔ Processed subject 0040008: shape = (150, 100)
✔ Processed subject 0040009: shape = (150, 100)


...

/home/cgntlab4/anaconda3/lib/python3.12/site-packages/nilearn/image/resampling.py:805: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  return resample_img(
/tmp/ipykernel_4090014/666536536.py:58: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  resampled_atlas = image.resample_to_img(atlas_img, subject_img, interpolation='nearest')
/home/cgntlab4/anaconda3/lib/python3.12/site-packages/nilearn/image/resampling.py:805: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  return resam

✔ Processed subject 0040143: shape = (150, 100)
✔ Processed subject 0040144: shape = (150, 100)
✔ Processed subject 0040145: shape = (150, 100)
✔ Processed subject 0040146: shape = (150, 100)
✔ Processed subject 0040147: shape = (150, 100)


/home/cgntlab4/anaconda3/lib/python3.12/site-packages/nilearn/image/resampling.py:805: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  return resample_img(
/tmp/ipykernel_4090014/666536536.py:58: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  resampled_atlas = image.resample_to_img(atlas_img, subject_img, interpolation='nearest')
/home/cgntlab4/anaconda3/lib/python3.12/site-packages/nilearn/image/resampling.py:805: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  return resam

7Networks_RH_Cont_PFCmp_1 ↔ 7Networks_LH_SomMot_4 | t = 4.394, p = 0.0000
7Networks_RH_Cont_PFCmp_1 ↔ 7Networks_RH_Cont_Par_1 | t = -4.288, p = 0.0000
7Networks_LH_Default_PFC_1 ↔ 7Networks_RH_Cont_Par_1 | t = -4.216, p = 0.0000
7Networks_LH_Default_PFC_4 ↔ 7Networks_LH_Limbic_TempPole_2 | t = -4.103, p = 0.0001
7Networks_RH_Default_PFCv_2 ↔ 7Networks_LH_SalVentAttn_FrOperIns_1 | t = -3.887, p = 0.0002
7Networks_RH_Default_PFCv_2 ↔ 7Networks_RH_SalVentAttn_Med_2 | t = -3.870, p = 0.0002
7Networks_LH_Default_PFC_1 ↔ 7Networks_RH_SalVentAttn_Med_2 | t = -3.840, p = 0.0002
7Networks_RH_Cont_PFCl_3 ↔ 7Networks_LH_SomMot_4 | t = 3.743, p = 0.0003
7Networks_RH_Default_PFCv_2 ↔ 7Networks_LH_SalVentAttn_Med_1 | t = -3.739, p = 0.0003
7Networks_RH_Cont_PFCl_4 ↔ 7Networks_RH_SomMot_6 | t = 3.622, p = 0.0004


/tmp/ipykernel_4090014/666536536.py:127: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0.
Use 'force_resample=True' to suppress this warning.
  resampled = image.resample_to_img(atlas_img, example_img, interpolation='nearest')
/home/cgntlab4/anaconda3/lib/python3.12/site-packages/nilearn/image/resampling.py:805: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  return resample_img(
